In [25]:
!pip install Flask
!pip install flask_ngrok
!pip install face_recognition
!pip install pyngrok==7.1.2
!ngrok authtoken 2fIxJpLgjs653mjOWz1NVxPGrN9_39Kd7f2WXEsfdX8K2fNnP

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [2]:
from flask_ngrok import run_with_ngrok
from flask import Flask
from pyngrok import ngrok
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import numpy as np
import cv2
import matplotlib.pyplot as plt
from torch.autograd import Variable
import time
import sys
from torch import nn
from torchvision import models
import face_recognition

In [3]:
im_size = 112
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]
sm = nn.Softmax()
inv_normalize =  transforms.Normalize(mean=-1*np.divide(mean,std),std=np.divide([1,1,1],std))
train_transforms = transforms.Compose([
                                        transforms.ToPILImage(),
                                        transforms.Resize((im_size,im_size)),
                                        transforms.ToTensor(),
                                        transforms.Normalize(mean,std)])


In [4]:
class Model(nn.Module):
    def __init__(self, num_classes,latent_dim= 2048, lstm_layers=1 , hidden_dim = 2048, bidirectional = False):
        super(Model, self).__init__()
        model = models.resnext50_32x4d(pretrained = True)
        self.model = nn.Sequential(*list(model.children())[:-2])
        self.lstm = nn.LSTM(latent_dim,hidden_dim, lstm_layers,  bidirectional)
        self.relu = nn.LeakyReLU()
        self.dp = nn.Dropout(0.4)
        self.linear1 = nn.Linear(2048,num_classes)
        self.avgpool = nn.AdaptiveAvgPool2d(1)
    def forward(self, x):
        batch_size,seq_length, c, h, w = x.shape
        x = x.view(batch_size * seq_length, c, h, w)
        fmap = self.model(x)
        x = self.avgpool(fmap)
        x = x.view(batch_size,seq_length,2048)
        x_lstm,_ = self.lstm(x,None)
        return fmap,self.dp(self.linear1(x_lstm[:,-1,:]))

In [5]:
class validation_dataset(Dataset):
    def __init__(self,video_names,sequence_length = 60,transform = None):
        self.video_names = video_names
        self.transform = transform
        self.count = sequence_length
    def __len__(self):
        return len(self.video_names)
    def __getitem__(self,idx):
        video_path = self.video_names[idx]
        frames = []
        a = int(100/self.count)
        first_frame = np.random.randint(0,a)
        for i,frame in enumerate(self.frame_extract(video_path)):
            #if(i % a == first_frame):
            faces = face_recognition.face_locations(frame)
            try:
              top,right,bottom,left = faces[0]
              frame = frame[top:bottom,left:right,:]
            except:
              pass
            frames.append(self.transform(frame))
            if(len(frames) == self.count):
              break
        #print("no of frames",len(frames))
        frames = torch.stack(frames)
        frames = frames[:self.count]
        return frames.unsqueeze(0)
    def frame_extract(self,path):
      vidObj = cv2.VideoCapture(path)
      success = 1
      while success:
          success, image = vidObj.read()
          if success:
              yield image
def im_convert(tensor):
    """ Display a tensor as an image. """
    image = tensor.to("cpu").clone().detach()
    image = image.squeeze()
    image = inv_normalize(image)
    image = image.numpy()
    image = image.transpose(1,2,0)
    image = image.clip(0, 1)
    cv2.imwrite('./2.png',image*255)
    return image


In [6]:
def im_plot(tensor):
    image = tensor.cpu().numpy().transpose(1,2,0)
    b,g,r = cv2.split(image)
    image = cv2.merge((r,g,b))
    image = image*[0.22803, 0.22145, 0.216989] +  [0.43216, 0.394666, 0.37645]
    image = image*255.0
    plt.imshow(image.astype(int))
    plt.show()

In [7]:

def predict(model,img,path = './'):
  fmap,logits = model(img.to('cuda'))
  params = list(model.parameters())
  weight_softmax = model.linear1.weight.detach().cpu().numpy()
  logits = sm(logits)
  _,prediction = torch.max(logits,1)
  confidence = logits[:,int(prediction.item())].item()*100
  print('confidence of prediction:',logits[:,int(prediction.item())].item()*100)
  idx = np.argmax(logits.detach().cpu().numpy())
  bz, nc, h, w = fmap.shape
  out = np.dot(fmap[-1].detach().cpu().numpy().reshape((nc, h*w)).T,weight_softmax[idx,:].T)
  predict = out.reshape(h,w)
  predict = predict - np.min(predict)
  predict_img = predict / np.max(predict)
  predict_img = np.uint8(255*predict_img)
  out = cv2.resize(predict_img, (im_size,im_size))
  heatmap = cv2.applyColorMap(out, cv2.COLORMAP_JET)
  img = im_convert(img[:,-1,:,:,:])
  result = heatmap * 0.5 + img*0.8*255
  cv2.imwrite('/content/1.png',result)
  result1 = heatmap * 0.5/255 + img*0.8
  r,g,b = cv2.split(result1)
  result1 = cv2.merge((r,g,b))
  plt.imshow(result1)
  plt.show()
  return [int(prediction.item()),confidence]

In [8]:
def prediction_result(video_path):
  im_size = 112
  mean=[0.485, 0.456, 0.406]
  std=[0.229, 0.224, 0.225]

  train_transforms = transforms.Compose([
                                          transforms.ToPILImage(),
                                          transforms.Resize((im_size,im_size)),
                                          transforms.ToTensor(),
                                          transforms.Normalize(mean,std)])


  path_to_videos= [video_path]
  flag = True
  video_dataset = validation_dataset(path_to_videos,sequence_length = 20,transform = train_transforms)

  model = Model(2).cuda()
  # path_to_model = '/content/drive/MyDrive/weights/model_90_acc_20_frames_FF_data.pt'
  path_to_model = '/content/drive/MyDrive/weights/checkpoint.pt'
  model.load_state_dict(torch.load(path_to_model))
  model.eval()
  for i in range(0,len(path_to_videos)):
    print(path_to_videos[i])
    prediction = predict(model,video_dataset[i],'./')
    if prediction[0] == 1:
      print("REAL")
      return "REAL"
    else:
      print("FAKE")
      return "FAKE"

In [9]:
from moviepy.editor import VideoFileClip

def convert_to_gif(input_file, output_file):
    clip = VideoFileClip(input_file)
    clip.write_gif(output_file)




/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:294: SyntaxWarning: invalid escape sequence '\d'
  lines_video = [l for l in lines if ' Video: ' in l and re.search('\d+x\d+', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:367: SyntaxWarning: invalid escape sequence '\d'
  rotation_lines = [l for l in lines if 'rotate          :' in l and re.search('\d+$', l)]
/usr/local/lib/python3.12/dist-packages/moviepy/video/io/ffmpeg_reader.py:370: SyntaxWarning: invalid escape sequence '\d'
  match = re.search('\d+$', rotation_line)
  if event.key is 'enter':



In [10]:
import cv2
import face_recognition
from google.colab.patches import cv2_imshow

def process_video(input_video_path, output_video_path, flag, duration_sec=4):

    cap = cv2.VideoCapture(input_video_path)


    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frames_to_process = min(int(fps * duration_sec), total_frames)


    fourcc = cv2.VideoWriter_fourcc(*'mp4v')


    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    label = "fake" if flag else "real"

    frame_count = 0
    while frame_count < frames_to_process:
        ret, frame = cap.read()
        if not ret:
            break


        rgb_frame = frame[:, :, ::-1]


        face_locations = face_recognition.face_locations(rgb_frame)


        for (top, right, bottom, left) in face_locations:
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
            cv2.putText(frame, label, (left, top - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 0, 255), 2)


        out.write(frame)


        cv2_imshow(frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_count += 1

    # Release resources
    cap.release()
    out.release()
    cv2.destroyAllWindows()

In [26]:
# ngrok.set_auth_token("2fIxJpLgjs653mjOWz1NVxPGrN9_39Kd7f2WXEsfdX8K2fNnP")
port=5000
ngrok.connect(port).public_url

'https://4e232623dfef.ngrok-free.app'

Ignore this

In [21]:
# @title Default title text
from google.colab.output import eval_js
print(eval_js("google.colab.kernel.proxyPort(5000)"))

https://5nspy7wsti-496ff2e9c6d22116-5000-colab.googleusercontent.com/


In [27]:
from flask import Flask ,  render_template , redirect, request, jsonify, send_from_directory , url_for
template_folder='/content/drive/MyDrive/template'
static_folder='/content/drive/MyDrive/static'
app = Flask(__name__,template_folder=template_folder,static_folder=static_folder)


@app.route("/" , methods=["GET"])
def home():
	try:
		return render_template("home.html")
	except Exception as e:
		return e

@app.route("/upload" , methods=["POST"])
def HandleUpload():
	try:

		f = request.files["video"]
		# first create static/videos then try to upload
		videoName = f"/content/drive/MyDrive/static/videos/{f.filename}"
		print(f)
		f.save(videoName)
		print("File saved!!!")
		# videoName=create_preprocessed_face_videos(videoName)

		# result_of_prediction=prediction_result(videoName)

		return redirect(f"/result-page/{f.filename}")
		# return redirect(f"/result/{True}")
	except Exception as e:
		print(e)
		return e

@app.route("/result-page/<string:fileName>" , methods=["GET"])
def Loader(fileName):
	try:
		print(fileName)
		return render_template("Result.html", path=f"videos/{fileName}", fileName=fileName, isLoading=True)
	except Exception as e:
		return e

@app.route("/predict" , methods=["GET"])
def restric():
	return redirect(url_for("home"))

@app.route("/predict/<string:fileName>" , methods=["GET"])
def Predict(fileName):
	try:
		print(fileName)
		videoName = f"/content/drive/MyDrive/static/videos/{fileName}"
		result_of_prediction=prediction_result(videoName)
		print(result_of_prediction)
		print("prediction completed")

		video_path = '/content/drive/MyDrive/static/videos/'
		input_video_path = os.path.join(video_path, fileName)
		output_video_path = os.path.join(video_path, 'output.mp4')
		flag = False if result_of_prediction == "REAL" else True
		process_video(input_video_path, output_video_path, flag, duration_sec=4)
		input_video = output_video_path
		output_gif = os.path.join(video_path, 'output.gif')
		convert_to_gif(input_video, output_gif)

		return jsonify({'result' : result_of_prediction == 'REAL', 'isLoading':False , 'fileName':"videos/output.gif"})
	except Exception as e:
		return e
# unused route
@app.route("/temp")
def temporary():
	try:
		return render_template("temp.html")
	except Exception as e:
		return e
# to be used route
@app.route("/result-vid/<string:result>" , methods=["GET"])
def Result(result,file):
	print("result page")
	print(result)
	if file:
		video_path = '/content/drive/MyDrive/static/videos/'
		file.save(os.path.join(video_path, file.filename))
		input_video_path = os.path.join(video_path, file.filename)
		outp="output_video.mp4"
		output_video_path = os.path.join(video_path, 'output.mp4')
		if result == "REAL":
			flag = False  # Set to False for "real"
		else:
			flag = True  # Set to True for "fake"

		process_video(input_video_path, output_video_path, flag, duration_sec=4)
		print('success')
    # Redirect to the view page with the filename as a query parameter
		input_video = "/content/output.mp4"
		output_gif = "/content/output.gif"
		convert_to_gif(input_video, output_gif)
		return redirect(url_for('view_video', filename='output.mp4'))

	return result




if __name__=="__main__":
  app.run(port=port)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
ERROR:__main__:Exception on / [GET]
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1511, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 920, in full_dispatch_request
    return self.finalize_request(rv)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 939, in finalize_request
    response = self.make_response(rv)
               ^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/flask/app.py", line 1249, in make_response
    raise TypeError(
TypeError: The view function did not return a valid response. The return type must be a string, di